In [1]:
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
from scipy.misc import imread, imresize
import numpy as np
import os

dataset_path = 'dataset/'
vectors_path = 'vectors/'
thumbs_path = 'thumbs/'
image_shape = (224, 224, 3)

Using TensorFlow backend.


In [2]:
# read dataset
with open('dataset.txt') as f:
    dataset = f.readlines()
dataset = [s.strip() for s in dataset]

In [3]:
len(dataset)

100800

In [5]:
# create dirs for vector files
vector_dirs = [vectors_path + os.path.dirname(s) for s in dataset]
vector_dirs = set(vector_dirs)
for d in vector_dirs:
    if not os.path.isdir(d):
        os.makedirs(d)
        
# create dirs for thumbs files
thumbs_dirs = [thumbs_path + os.path.dirname(s) for s in dataset]
thumbs_dirs = set(thumbs_dirs)
for d in thumbs_dirs:
    if not os.path.isdir(d):
        os.makedirs(d)

In [6]:
feature_extractor = ResNet50(include_top=False, weights='imagenet', input_shape=image_shape)

In [7]:
def read_resize(fn):
    img = imread(dataset_path + fn, mode='RGB')
    img = imresize(img, image_shape)
    return img

pool = ThreadPool(cpu_count())

In [ ]:
batch_size = 64

# read dataset in batches
for i in range(0, len(dataset), batch_size):
    batch = dataset[i: i+batch_size]
    # load/resize images in parallel threads
    x = pool.map(read_resize, batch)
    # preprocess
    x = np.array(x, dtype='float32')
    x = preprocess_input(x)
    # extract feature vectors
    y = feature_extractor.predict(x)
    # save vectors
    for j, fn in enumerate(batch): 
        np.save(vectors_path + fn, y[j][0, 0])
    
    if not i % (batch_size*16):
        print(i)

(64, 1, 1, 2048)
[[[ 0.0022143   0.05169188  0.53710043 ...,  0.          0.          0.        ]]]
[[[ 0.          0.0125021   0.         ...,  0.03175412  0.33210948
    0.05932942]]]
[[[ 0.11665213  0.01732102  0.20555854 ...,  0.09217698  0.38710591  0.        ]]]
[[[ 0.18200326  0.          0.17952576 ...,  1.71178198  0.17060593
    0.05263219]]]
[[[ 0.          0.17657079  0.26027352 ...,  0.15416838  0.57855558
    0.12220415]]]
[[[ 0.          0.24481741  0.04946096 ...,  0.          0.3756963
    0.22985591]]]
[[[ 0.03636662  0.40350261  0.0291154  ...,  0.07698201  0.          0.        ]]]
[[[ 0.15446775  0.0600804   0.         ...,  0.07835966  0.56663287
    0.44229174]]]
[[[ 0.00730159  0.07878184  0.15795937 ...,  0.05200006  0.01420016
    0.26751947]]]
[[[ 0.10149877  0.15563487  0.         ...,  0.64658213  0.08828382  0.        ]]]
[[[ 0.21094562  0.02625934  0.76445282 ...,  0.02276522  0.46049982
    0.25167188]]]
[[[ 0.24887779  0.06350359  0.36858678 ...,  1.225

(64, 1, 1, 2048)
[[[ 0.          0.79862642  0.87802857 ...,  0.36389005  0.07100878
    0.59539455]]]
[[[ 0.40528458  1.70000339  0.         ...,  0.03940062  0.12626453
    0.23628388]]]
[[[ 0.00637159  0.28313377  0.         ...,  0.16896036  0.17928281
    0.0239017 ]]]
[[[ 0.          0.0880143   0.15141836 ...,  0.06516545  0.65939611
    1.03882253]]]
[[[ 0.03148162  0.02541628  0.         ...,  0.13657415  0.          0.        ]]]
[[[ 0.20763673  0.43815029  0.67347765 ...,  0.26968288  0.00121105
    0.17875506]]]
[[[ 0.19213125  0.00145759  0.         ...,  0.06756587  0.          0.47317311]]]
[[[ 0.03597138  0.00164819  0.67988873 ...,  0.58139086  0.11522587
    0.0125087 ]]]
[[[ 0.          0.09115315  0.00020507 ...,  0.02943387  0.18922038
    0.06016236]]]
[[[ 0.          0.03313566  0.63520068 ...,  0.0158777   0.          0.        ]]]
[[[ 0.          0.32555017  0.24157026 ...,  0.32668951  0.          0.24415453]]]
[[[ 0.39813474  0.52557731  0.20540416 ...,  0.  

(64, 1, 1, 2048)
[[[ 0.02697945  1.09009027  0.00256012 ...,  0.          0.          0.13589017]]]
[[[ 0.01156827  0.19777189  0.11488891 ...,  0.          0.07453939
    0.42328247]]]
[[[ 0.          0.          0.81581801 ...,  0.07683677  0.          0.12322595]]]
[[[ 0.11133993  0.04000089  0.12917218 ...,  0.00956466  0.08624154  0.        ]]]
[[[ 0.          0.57656157  0.14218603 ...,  0.          0.02251494
    0.41475776]]]
[[[ 0.00732327  0.34683263  0.02284748 ...,  0.20992844  0.00940376
    1.35068727]]]
[[[  3.70663940e-04   3.15109640e-02   6.20537400e-02 ...,   4.20110561e-02
     8.06927204e-01   6.69443548e-01]]]
[[[ 0.          0.01816166  0.27086782 ...,  0.23332508  0.13875431
    0.00074621]]]
[[[ 0.          0.00221512  0.10851046 ...,  0.00334648  0.01047042
    1.47209048]]]
[[[ 0.          0.3181065   0.02001299 ...,  0.13096988  0.06058552
    0.37322563]]]
[[[ 0.10405625  0.13832438  0.04402061 ...,  0.          0.          0.17876834]]]
[[[ 0.          0.1

(64, 1, 1, 2048)
[[[ 0.04011279  0.92074376  0.61484188 ...,  0.0689914   0.08688577
    0.77297747]]]
[[[ 0.23996389  0.          0.82609218 ...,  0.14283726  0.00577765
    0.22665042]]]
[[[  1.25033960e-01   1.91212252e-01   2.58541841e-04 ...,   2.96018392e-01
     0.00000000e+00   1.36368960e-01]]]
[[[ 0.04436587  0.10014743  0.         ...,  0.05153064  0.10130265
    0.15632901]]]
[[[ 0.          1.52313042  0.0532122  ...,  0.00972276  0.02601957
    0.69561768]]]
[[[ 0.          0.05081658  0.7502315  ...,  0.          0.00234992
    0.00509097]]]
[[[  5.87268732e-04   3.81290823e-01   0.00000000e+00 ...,   0.00000000e+00
     1.52439177e+00   0.00000000e+00]]]
[[[ 0.01482938  0.21155638  0.20540968 ...,  0.32008171  0.36029944
    0.00134788]]]
[[[ 0.87461513  0.15024164  0.45875183 ...,  1.29140949  0.01882719
    0.3112298 ]]]
[[[ 0.00338264  0.25387716  0.         ...,  0.          0.17574731
    0.01730088]]]
[[[ 0.00868469  0.28188938  0.01265113 ...,  0.11117627  0.    

(64, 1, 1, 2048)
[[[ 0.09658945  0.04186722  0.11308187 ...,  0.03523349  0.24297029
    0.13938552]]]
[[[ 0.26153025  0.          0.02148937 ...,  0.02897885  0.10986763
    0.00318941]]]
[[[ 0.10468677  1.55089736  0.04302101 ...,  0.06360815  0.46914938
    0.98343509]]]
[[[ 0.          0.13140781  0.05069445 ...,  0.12612522  0.33412161
    0.03091834]]]
[[[ 0.          0.09056876  0.0167113  ...,  0.59126544  0.52448219
    0.51120228]]]
[[[ 0.07200349  0.77770418  0.1537331  ...,  0.          0.10911076
    0.07421493]]]
[[[ 0.26555738  0.70515144  0.23076542 ...,  0.03055677  0.02155906
    0.4161565 ]]]
[[[ 0.36346322  0.3535997   0.24997661 ...,  0.07804251  0.78014427
    0.33088142]]]
[[[ 0.00136404  0.          0.07036799 ...,  0.47251704  0.3528116
    0.15732798]]]
[[[ 0.08128979  0.74801016  0.11895052 ...,  0.01137487  0.3574388
    0.71311545]]]
[[[ 0.02569823  0.60517591  0.0894096  ...,  0.05352072  0.5318675
    0.18240823]]]
[[[ 0.06371944  0.6666044   0.02237386 .

...

(64, 1, 1, 2048)
[[[ 1.08006287  0.74469447  0.66951704 ...,  0.29390591  0.00600205
    0.13341272]]]
[[[ 0.51196122  1.75268352  1.03464317 ...,  0.4316709   0.          0.67777562]]]
[[[ 0.20167534  1.91910005  0.00486941 ...,  1.13577986  0.45466903
    0.53349286]]]
[[[ 1.64282727  0.36076722  0.00763508 ...,  0.36076114  1.42892301
    0.12411688]]]
[[[ 1.11507833  0.60740542  0.00487226 ...,  1.19657695  0.09042408
    0.59980029]]]
[[[ 0.36700296  2.46495175  0.13070324 ...,  0.35048816  1.6495856
    0.18940143]]]
[[[ 0.93976253  0.20241211  1.54532778 ...,  0.48224288  1.76590228
    0.00204822]]]
[[[ 0.08510029  3.00958538  1.6124866  ...,  1.16841137  0.08147765
    1.01617944]]]
[[[ 0.23897383  2.26871729  0.20439675 ...,  4.16753435  0.4236486
    0.06441129]]]
[[[ 0.00606697  0.85692132  0.33312631 ...,  0.1098095   0.15893275
    2.42238545]]]
[[[ 0.83515954  1.89625204  0.4845652  ...,  1.14467907  0.33013448
    0.74160862]]]
[[[ 0.32992321  0.15332599  0.02915171 ...

(64, 1, 1, 2048)
[[[ 0.01455859  2.14117169  0.33853874 ...,  0.81424814  0.          0.14008471]]]
[[[ 0.67978919  0.31846452  0.12421151 ...,  0.29583102  0.95126671  0.        ]]]
[[[ 0.00747184  0.50195432  0.00563605 ...,  0.83929163  0.71155375
    0.01744449]]]
[[[ 0.07220647  1.11329699  0.35410383 ...,  0.40153581  0.53293765
    1.64783561]]]
[[[ 0.04344342  0.39904529  0.04092688 ...,  0.19519734  0.81123632
    0.66859776]]]
[[[ 0.03130316  0.09200364  0.06739395 ...,  0.95150727  0.24273382
    0.29938841]]]
[[[  8.32720324e-02   1.39021969e+00   2.91688106e-04 ...,   3.11547577e-01
     1.42109382e+00   1.72094658e-01]]]
[[[ 0.06933761  2.56009984  0.         ...,  0.02357459  0.62250209
    0.21705881]]]
[[[ 0.          0.06890433  0.03159851 ...,  0.77600604  0.35931388
    0.17348488]]]
[[[ 0.00803074  3.21261382  0.14678638 ...,  0.30585751  0.48944667
    0.16160128]]]
[[[ 0.1802568   0.60827601  0.06473888 ...,  0.55571252  1.27153277
    0.40566128]]]
[[[ 0.5553973

(64, 1, 1, 2048)
[[[ 0.03001073  0.19598632  0.         ...,  0.20083378  0.00058267  0.        ]]]
[[[ 0.26263353  0.09357227  0.13696274 ...,  0.84658939  0.41754776
    0.11977576]]]
[[[ 0.04019107  0.04807822  0.00875848 ...,  1.32006288  0.30084524
    0.10899504]]]
[[[ 0.05653362  0.17447753  0.         ...,  0.03621743  0.          0.53906715]]]
[[[ 0.04462508  0.          0.00174629 ...,  0.54846746  0.20217788
    0.17171803]]]
[[[ 0.          0.45079291  0.02310421 ...,  0.68058926  0.08468402
    0.15292804]]]
[[[ 0.          1.08833194  0.         ...,  0.51163447  0.          0.0079051 ]]]
[[[ 0.10172031  0.50588095  0.30049056 ...,  1.24030471  0.30851412
    0.92430502]]]
[[[ 0.01398457  0.26499864  0.2017027  ...,  0.07055193  0.08943972
    0.61456138]]]
[[[ 0.07727692  0.83440655  0.         ...,  0.33015472  0.00634685  0.        ]]]
[[[ 0.07937159  0.66637111  0.         ...,  0.00657021  0.03944947  0.        ]]]
[[[ 0.07172582  2.28744888  0.02621893 ...,  0.13540

(64, 1, 1, 2048)
[[[ 0.          0.27601409  0.63515955 ...,  0.48781055  0.46399096
    0.14825861]]]
[[[ 0.00631161  0.36962712  0.42061517 ...,  1.92919242  0.37601182
    0.08352628]]]
[[[ 0.24500057  0.06835626  0.08892734 ...,  0.96522498  0.21286385
    0.05682308]]]
[[[ 0.01311322  0.10730534  0.03784528 ...,  0.02249428  0.93665135
    0.25277421]]]
[[[ 0.          0.97313255  0.         ...,  0.32118171  0.04342712
    0.12535235]]]
[[[ 0.56691945  0.          0.20991473 ...,  2.61646414  1.13912559
    0.01332695]]]
[[[ 0.          0.01003502  0.06515425 ...,  0.12128423  0.14474547
    0.02221915]]]
[[[  6.33389410e-03   1.86995000e-01   1.70923322e-02 ...,   6.43688965e+00
     1.76390827e-01   6.90189749e-02]]]
[[[ 0.03089119  0.16998957  0.         ...,  0.3646417   0.05307876
    0.0075756 ]]]
[[[  2.10166764e+00   0.00000000e+00   1.16807592e+00 ...,   9.84763131e-02
     1.80204061e-03   1.60414135e+00]]]
[[[ 0.28553379  0.12763067  0.         ...,  2.30886292  0.1050

KeyboardInterrupt: 